In [1]:
# Get imports from imports.py
# Will try to pull imports from ./../modules/
from imports import *

In [2]:
input_dir1 = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/hst_contsub/'
input_dir2 = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628e/hst_contsub/'
output_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628mosaic/hst_contsub/'

input_file1 = glob(f'{input_dir1}/*_hst_ha_as2.fits')[0]
input_file1_err = glob(f'{input_dir1}/*_hst_ha_err_as2.fits')[0]

input_file2 = glob(f'{input_dir2}/*_hst_ha_as2.fits')[0]
input_file2_err = glob(f'{input_dir2}/*_hst_ha_err_as2.fits')[0]

print(input_file1)
print(input_file2)
print(input_file1_err)
print(input_file2_err)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/hst_contsub/ngc628c_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628e/hst_contsub/ngc628e_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/hst_contsub/ngc628c_hst_ha_err_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628e/hst_contsub/ngc628e_hst_ha_err_as2.fits


In [3]:
hdu1_img = fits.open(input_file1)
hdu1_err = fits.open(input_file1_err)

hdu2_img = fits.open(input_file2)
hdu2_err = fits.open(input_file2_err)

In [4]:
wcs_out, shape_out = find_optimal_celestial_wcs([hdu1_img, hdu2_img])

array_bgmatch, _ = reproject_and_coadd([hdu1_img, hdu2_img],
                                       wcs_out, 
                                       shape_out=shape_out,
                                       reproject_function=reproject_interp,
                                        match_background=True)

array_err, _ = reproject_and_coadd([hdu1_err, hdu2_err],
                                            wcs_out, 
                                            shape_out=shape_out,
                                            reproject_function=reproject_interp,
                                            match_background=False)

In [5]:
keys = ['NAXIS1', 'NAXIS2', 'CRPIX1', 'CRPIX2', 'CRVAL1', 'CRVAL2']
hdr_new = hdu1_img[0].header
for key in keys:
    if 'NAXIS1' in key:
        hdr_new[key] = array_bgmatch.shape[1]
    elif 'NAXIS2' in key:
        hdr_new[key] = array_bgmatch.shape[0]
    else:
        hdr_new[key] = wcs_out.to_header()[key]

In [6]:
hdu_mosaic_bgmatch = fits.PrimaryHDU(data=array_bgmatch, header=hdr_new)
hdu_mosaic_bgmatch.writeto('/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628mosaic/hst_contsub/ngc628mosaic_hst_ha_as2.fits', overwrite=True)

hdu_mosaic_bgmatch_err = fits.PrimaryHDU(data=array_err, header=hdr_new)
hdu_mosaic_bgmatch_err.writeto('/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628mosaic/hst_contsub/ngc628mosaic_hst_ha_err_as2.fits', overwrite=True)